# Revenue Analysis Across Mobile Plans for a Telecom Operator

**Data Provided**: A sample of 500 users from a federal mobile network operator, including user demographics (who they are, location, plan type) and their usage data (calls, messages, and internet traffic throughout 2018).

**Research Objective**: To analyze customer usage patterns of telecommunication services and determine which plan generates higher revenue, enabling optimized advertising budget allocation.

In [1]:
from pathlib import Path
import pandas as pd
import warnings
import numpy as np

def configure_display_settings():
    """
    Configure pandas and numpy display settings for optimal readability.
    
    Settings:
    - Suppresses warnings
    - Disables scientific notation
    - Shows all columns
    - Sets consistent decimal formatting
    """
    # Suppress warnings
    warnings.filterwarnings('ignore')
    
    # Pandas display options
    # Disable scientific notation
    pd.set_option('display.float_format', '{:.2f}'.format)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.precision', 2)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
    # Numpy display options
    np.set_printoptions(suppress=True, precision=2)
    
    print("✓ Display settings configured")

# Apply configuration
configure_display_settings()

✓ Display settings configured


## Data Loading and Initial Exploration

This project utilizes 5 datasets. Let's load and examine each one:

In [2]:
# Define base data path
DATA_PATH = Path('./data/yandex/telecom')

# Load datasets
calls = pd.read_csv(DATA_PATH / 'calls.csv')
internet = pd.read_csv(DATA_PATH / 'internet.csv', index_col=0)
messages = pd.read_csv(DATA_PATH / 'messages.csv')
tariffs = pd.read_csv(DATA_PATH / 'tariffs.csv')
users = pd.read_csv(DATA_PATH / 'users.csv')

**Note on `Unnamed: 0` column** in the internet dataset:

The `Unnamed: 0` column appears when a DataFrame is saved with an index using `pd.DataFrame().to_csv(..., index=True)`. This column can be handled in two ways:

1. **During reading (recommended):** Specify it as the index when loading

2. **After reading:** Drop it manually:
```python
   internet = internet.drop(columns=['Unnamed: 0'])
```
---

Let's create a helper function to display initial data overview and quality metrics:

In [3]:
def show_info(data: pd.DataFrame, name: str = "Dataset") -> None:
    """
    Display initial exploration metrics for a DataFrame.
    
    Parameters:
    -----------
    data : pd.DataFrame
        The dataset to explore
    name : str
        Name of the dataset for display purposes
    """
    print(f"\n{'='*60}")
    print(f"{name.upper()} - Initial Exploration")
    print(f"{'='*60}\n")
    
    # Display first few rows
    display(data.head())
    
    # Show structure and data types
    print("\nDataset Info:")
    data.info()
    
    # Check for duplicates
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html
    # By default use all of the columns. Use subset parameter to specify particular columns.
    n_duplicates = data.duplicated().sum()
    print(f"\nDuplicate rows: {n_duplicates}")
    
    # Additional quality checks
    print(f"Missing values per column:")
    missing = data.isnull().sum()
    if missing.sum() > 0:
        display(missing[missing > 0])
    else:
        print("No missing values found")
    
    print(f"\nDataset shape: {data.shape[0]:,} rows × {data.shape[1]} columns")

In [4]:
def check_data_quality(df, categorical_cols, numerical_cols=None):
    """
    Perform comprehensive data quality checks on specified columns.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Dataset to check
    categorical_cols : list
        Columns to check for unique values and anomalies
    numerical_cols : list, optional
        Columns to check for range/outliers

    -----------
    This helper function was developed with Gen AI assistance. There are several comments 
    to explain the logic and checks being performed:
    
    1) When to use \n:
        - At the start of a string when you want a blank line before that print
        - At the end of a string when you want a blank line after that print
        - Use print() alone for a single blank line (cleaner and more explicit)
        This is such GenAI solution but it improves the readability of the output.

    2) Deliberate use of emodjis:
        - "⚠️" for warnings about potential data issues
        - "ℹ️" for informational messages about data characteristics
        - "✓" for successful completion messages
        Another hallmark of GenAI solutions but again I feel it helps to spot 
        problem lines quickly when you glance over the output.

    TO DO: 
        - sanity check for date_time dtype columns

    """
    print("DATA QUALITY CHECKS")
    print("=" * 60)
    
    # Categorical columns
    if categorical_cols:
        print("\nCATEGORICAL COLUMNS:")
        print("-" * 60)
        for col in categorical_cols:
            n_unique = df[col].nunique()
            n_missing = df[col].isnull().sum()
            
            print(f"\n{col.upper()}:")
            print(f"  • Unique values: {n_unique}")
            print(f"  • Sample values: {list(df[col].unique()[:5])}")
            
            # Check for missing values
            if n_missing > 0:
                pct_missing = (n_missing / len(df)) * 100
                print(f"  ⚠️  Missing values: {n_missing} ({pct_missing:.1f}%)")
            
            # Check for potential issues
            if n_unique == 1:
                print(f"  ⚠️  Warning: Only 1 unique value (constant column)")
            elif n_unique == len(df):
                print(f"  ⚠️  Warning: All values unique (potential identifier)")
            
            # Check for empty strings or whitespace
            if df[col].dtype == 'object':
                empty_strings = (df[col].str.strip() == '').sum()
                if empty_strings > 0:
                    print(f"  ⚠️  Empty/whitespace strings: {empty_strings}")
                
                # Check for unusual characters or patterns
                if df[col].str.contains(r'[^\w\s-]', na=False, regex=True).any():
                    print(f"  ℹ️  Contains special characters")
    
    # Numerical columns
    if numerical_cols:
        print(f"\n\nNUMERICAL COLUMNS:")
        print("-" * 60)
        display(df[numerical_cols].describe())
        
        # Check for anomalies in each numerical column
        for col in numerical_cols:
            n_missing = df[col].isnull().sum()
            min_val = df[col].min()
            max_val = df[col].max()
            mean_val = df[col].mean()
            median_val = df[col].median()
            
            print(f"\n{col.upper()} - Anomaly Checks:")
            
            # Missing values
            if n_missing > 0:
                pct_missing = (n_missing / len(df)) * 100
                print(f"  ⚠️  Missing values: {n_missing} ({pct_missing:.1f}%)")
            
            # Negative values
            if min_val < 0:
                n_negative = (df[col] < 0).sum()
                print(f"  ⚠️  Negative values: {n_negative} (min: {min_val})")
            
            # Age-specific checks
            if 'age' in col.lower():
                if min_val < 10:
                    print(f"  ⚠️  Unusually young age detected: {min_val}")
                if max_val > 100:
                    print(f"  ⚠️  Unusually old age detected: {max_val}")
                if min_val < 18:
                    print(f"  ℹ️  Contains minors (age < 18)")
            
            # Check for outliers using IQR method
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
            
            if len(outliers) > 0:
                pct_outliers = (len(outliers) / len(df)) * 100
                print(f"  ℹ️  Potential outliers (IQR method): {len(outliers)} ({pct_outliers:.1f}%)")
            
            # Check if mean and median are very different (skewness indicator)
            if abs(mean_val - median_val) > 0.5 * median_val:
                print(f"  ℹ️  Large mean-median difference: mean={mean_val:.2f}, median={median_val:.2f} (skewed distribution)")
    
    print("\n" + "=" * 60)
    print("✓ Quality check complete")

### 1. Calls Dataset

In [5]:
show_info(calls, "Calls Dataset")



CALLS DATASET - Initial Exploration



,id,call_date,duration,user_id
0,1000_0,2018-07-25,0.00,1000
1,1000_1,2018-08-17,0.00,1000
2,1000_2,2018-06-11,2.85,1000
3,1000_3,2018-09-21,13.80,1000
4,1000_4,2018-12-15,5.18,1000



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202607 entries, 0 to 202606
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         202607 non-null  object 
 1   call_date  202607 non-null  object 
 2   duration   202607 non-null  float64
 3   user_id    202607 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 6.2+ MB

Duplicate rows: 0
Missing values per column:
No missing values found

Dataset shape: 202,607 rows × 4 columns


**1. Calls Dataset**  

**Description:** Contains information about phone calls made by users.

| Column | Description | Data Type | Notes |
|--------|-------------|-----------|-------|
| `id` | Unique call identifier | int | Rename for clarity |
| `call_date` | Date of the call | object → datetime | Requires type conversion |
| `duration` | Call duration in minutes | float → int | Will be rounded up to whole minutes per company policy |
| `user_id` | User identifier who made the call | int | Foreign key to users table |

**Data Quality Notes:**
- ✓ No missing values detected  
- 18277 full duplicate rows  
- ⚠️ `call_date` needs conversion to datetime format
- ⚠️ `duration` should be rounded to integers (company rounds even 1-second calls to full minutes)

In [6]:
# Data cleaning: Calls dataset
calls = calls.rename(columns={'id': 'call_id'})  # Rename for clarity
calls['call_date'] = pd.to_datetime(calls['call_date'], format='%Y-%m-%d')
calls['duration'] = np.ceil(calls['duration']).astype(int)  # Round up and convert to integer

In [7]:
# TO DO - think about sanity check for date_time dtype columns

check_data_quality(
    df=calls,
    categorical_cols=['user_id'],
    numerical_cols=['duration']
)

DATA QUALITY CHECKS

CATEGORICAL COLUMNS:
------------------------------------------------------------

USER_ID:
  • Unique values: 492
  • Sample values: [np.int64(1000), np.int64(1001), np.int64(1002), np.int64(1003), np.int64(1004)]


NUMERICAL COLUMNS:
------------------------------------------------------------


,duration
count,202607.00
mean,7.16
std,5.96
min,0.00
25%,2.00
50%,6.00
75%,11.00
max,38.00



DURATION - Anomaly Checks:
  ℹ️  Potential outliers (IQR method): 1041 (0.5%)

✓ Quality check complete


### 2. Users Dataset

In [8]:
show_info(users, "Users Dataset")


USERS DATASET - Initial Exploration



,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
0,1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1,1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart
2,1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart
3,1003,23,NaN,Москва,Белла,Белякова,2018-08-17,ultra
4,1004,68,NaN,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   age         500 non-null    int64 
 2   churn_date  38 non-null     object
 3   city        500 non-null    object
 4   first_name  500 non-null    object
 5   last_name   500 non-null    object
 6   reg_date    500 non-null    object
 7   tariff      500 non-null    object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB

Duplicate rows: 0
Missing values per column:


churn_date    462
dtype: int64


Dataset shape: 500 rows × 8 columns


**2. Users Dataset**

**Description:** Contains user demographic and subscription information.

| Column | Description | Data Type | Notes |
|--------|-------------|-----------|-------|
| `user_id` | Unique user identifier | int | Primary key |
| `first_name` | User's first name | object | |
| `last_name` | User's last name | object | |
| `age` | User's age in years | int | |
| `reg_date` | Plan activation date | object → datetime | Requires type conversion |
| `churn_date` | Plan cancellation date | object → datetime | NULL if plan still active |
| `city` | User's city of residence | object | |
| `tariff` | Tariff plan name | object | Foreign key to tariffs table |

**Data Quality Notes:**
- ✓ Missing values in `churn_date` are expected (active subscriptions)
- ⚠️ Date columns need conversion to datetime format
- ✓ All other columns are clean


In [9]:
# Apply to users dataset
check_data_quality(
    users, 
    categorical_cols=['user_id', 'first_name', 'last_name', 'city', 'tariff'],
    numerical_cols=['age']
)

DATA QUALITY CHECKS

CATEGORICAL COLUMNS:
------------------------------------------------------------

USER_ID:
  • Unique values: 500
  • Sample values: [np.int64(1000), np.int64(1001), np.int64(1002), np.int64(1003), np.int64(1004)]
  ⚠️  Warning: All values unique (potential identifier)

FIRST_NAME:
  • Unique values: 246
  • Sample values: ['Рафаил', 'Иван', 'Евгений', 'Белла', 'Татьяна']

LAST_NAME:
  • Unique values: 419
  • Sample values: ['Верещагин', 'Ежов', 'Абрамович', 'Белякова', 'Авдеенко']

CITY:
  • Unique values: 76
  • Sample values: ['Краснодар', 'Москва', 'Стерлитамак', 'Новокузнецк', 'Набережные Челны']

TARIFF:
  • Unique values: 2
  • Sample values: ['ultra', 'smart']


NUMERICAL COLUMNS:
------------------------------------------------------------


,age
count,500.00
mean,46.59
std,16.67
min,18.00
25%,32.00
50%,46.00
75%,62.00
max,75.00



AGE - Anomaly Checks:

✓ Quality check complete


- We have 492 unique user IDs in the calls dataset and 500 in the users dataset. 

In [10]:
users['churn_date'] = pd.to_datetime(users['churn_date'], format='%Y-%m-%d')

users['reg_date'] = pd.to_datetime(users['reg_date'], format='%Y-%m-%d')

### 3. Internet Session Dataset

In [11]:
show_info(internet, "Internet Usage Dataset")


INTERNET USAGE DATASET - Initial Exploration



,id,mb_used,session_date,user_id
0,1000_0,112.95,2018-11-25,1000
1,1000_1,1052.81,2018-09-07,1000
2,1000_2,1197.26,2018-06-25,1000
3,1000_3,550.27,2018-08-22,1000
4,1000_4,302.56,2018-09-24,1000



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 149396 entries, 0 to 149395
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            149396 non-null  object 
 1   mb_used       149396 non-null  float64
 2   session_date  149396 non-null  object 
 3   user_id       149396 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 5.7+ MB

Duplicate rows: 0
Missing values per column:
No missing values found

Dataset shape: 149,396 rows × 4 columns


**3. Internet Sessions Dataset**  

**Description:** Contains information about internet usage sessions.

| Column | Description | Data Type | Notes |
|--------|-------------|-----------|-------|
| `id` | Unique session identifier | int |  |
| `mb_used` | Data consumed per session (MB) | float | |
| `session_date` | Date of internet session | object → datetime | Requires type conversion |
| `user_id` | User identifier | int | Foreign key to users table |

**Data Quality Notes:**
- ✓ No missing values detected
- ⚠️ `session_date` needs conversion to datetime format

In [12]:
internet['session_date'] = pd.to_datetime(internet['session_date'], format ='%Y-%m-%d')

internet = internet.rename(columns={'id': 'session_id'})  # Rename for clarity

### 4. Messages Dataset

In [13]:
show_info(messages, "Messages Dataset")


MESSAGES DATASET - Initial Exploration



,id,message_date,user_id
0,1000_0,2018-06-27,1000
1,1000_1,2018-10-08,1000
2,1000_2,2018-08-04,1000
3,1000_3,2018-06-16,1000
4,1000_4,2018-12-05,1000



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123036 entries, 0 to 123035
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            123036 non-null  object
 1   message_date  123036 non-null  object
 2   user_id       123036 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.8+ MB

Duplicate rows: 0
Missing values per column:
No missing values found

Dataset shape: 123,036 rows × 3 columns


In [14]:
check_data_quality(df=messages, 
    categorical_cols=['user_id', 'id'])

DATA QUALITY CHECKS

CATEGORICAL COLUMNS:
------------------------------------------------------------

USER_ID:
  • Unique values: 426
  • Sample values: [np.int64(1000), np.int64(1002), np.int64(1003), np.int64(1004), np.int64(1005)]

ID:
  • Unique values: 123036
  • Sample values: ['1000_0', '1000_1', '1000_2', '1000_3', '1000_4']
  ⚠️  Warning: All values unique (potential identifier)

✓ Quality check complete


**4. Messages Dataset**  

**Description:** Contains information about SMS/text messages sent by users.

| Column | Description | Data Type | Notes |
|--------|-------------|-----------|-------|
| `id` | Unique message identifier | int | Rename to the message_id |
| `message_date` | Date message was sent | object → datetime | Requires type conversion |
| `user_id` | User identifier who sent message | int | Foreign key to users table |

**Data Quality Notes:**
- ✓ No missing values detected
- ⚠️ `message_date` needs conversion to datetime format
- `id` column will be renamed to 'message_id'

In [15]:
# Convert date columns to datetime
messages['message_date'] = pd.to_datetime(messages['message_date'], format='%Y-%m-%d')
# Verify conversion
print(f"✓ message_date converted to {messages['message_date'].dtype}")

messages = messages.rename(columns={'id': 'message_id'})  # Rename for clarity

✓ message_date converted to datetime64[ns]


### 5. Tariffs Dataset

In [16]:
show_info(tariffs, "Tariffs Dataset")


TARIFFS DATASET - Initial Exploration



,messages_included,mb_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,tariff_name
0,50,15360,500,550,200,3,3,smart
1,1000,30720,3000,1950,150,1,1,ultra



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   messages_included      2 non-null      int64 
 1   mb_per_month_included  2 non-null      int64 
 2   minutes_included       2 non-null      int64 
 3   rub_monthly_fee        2 non-null      int64 
 4   rub_per_gb             2 non-null      int64 
 5   rub_per_message        2 non-null      int64 
 6   rub_per_minute         2 non-null      int64 
 7   tariff_name            2 non-null      object
dtypes: int64(7), object(1)
memory usage: 260.0+ bytes

Duplicate rows: 0
Missing values per column:
No missing values found

Dataset shape: 2 rows × 8 columns


**5. Tariffs Dataset**  

**Description:** Contains tariff plan details and pricing structure.

| Column | Description | Data Type | Notes |
|--------|-------------|-----------|-------|
| `tariff_name` | Name of the tariff plan | object | Primary key |
| `rub_monthly_fee` | Monthly subscription fee (RUB) | float | |
| `minutes_included` | Included minutes per month | int | |
| `messages_included` | Included messages per month | int | |
| `mb_per_month_included` | Included data per month (MB) | int | |
| `rub_per_minute` | Cost per additional minute (RUB) | float | Charged when exceeding included minutes |
| `rub_per_message` | Cost per additional message (RUB) | float | Charged when exceeding included messages |
| `rub_per_gb` | Cost per additional GB (RUB) | float | Charged when exceeding included data |

**Data Quality Notes:**
- ✓ No missing values detected
- ✓ All columns are in correct format


### Data Relationships

users (1) ----< (M) calls  
users (1) ----< (M) internet  
users (1) ----< (M) messages  
users (M) >---- (1) tariffs  

## Step 2. Monthly Aggregation  

Revenue analysis for telecom operators requires monthly aggregation for several strategic reasons:  
- Tariff plans operate on monthly billing cycles  
- Strategic decisions (marketing budget allocation, tariff adjustments) operate on monthly or quarterly timescales  
- Daily fluctuations in usage are noise for strategic analysis  
- Data volume management:  
    - Reduces dataset size while preserving analytical value  
    - Individual call/session records: ~200k+ rows
    - Monthly user aggregations: ~6,000 rows (500 users × 12 months)
97% reduction in data volume with no loss of business insights

Step 1: extract time period

In [17]:
# Extract year-month period for grouping
# Using Period ensures months are treated correctly across years
calls['period'] = calls['call_date'].dt.to_period('M')
internet['period'] = internet['session_date'].dt.to_period('M')
messages['period'] = messages['message_date'].dt.to_period('M')

In [18]:
# Verify period extraction
print(f"Date range in data:\n")
print(f"  Calls: {calls['call_date'].min()} to {calls['call_date'].max()}")
print(f"  Internet: {internet['session_date'].min()} to {internet['session_date'].max()}")
print(f"  Messages: {messages['message_date'].min()} to {messages['message_date'].max()}\n")

# Verify period extraction
print(f"Period range in data:\n")
print(f"  Calls: {calls['period'].min()} to {calls['period'].max()}")
print(f"  Internet: {internet['period'].min()} to {internet['period'].max()}")
print(f"  Messages: {messages['period'].min()} to {messages['period'].max()}")


Date range in data:

  Calls: 2018-01-01 00:00:00 to 2018-12-31 00:00:00
  Internet: 2018-01-01 00:00:00 to 2018-12-31 00:00:00
  Messages: 2018-01-02 00:00:00 to 2018-12-31 00:00:00

Period range in data:

  Calls: 2018-01 to 2018-12
  Internet: 2018-01 to 2018-12
  Messages: 2018-01 to 2018-12


Why to use Period pandas object:  
 - Encodes year+month as a single period object (e.g. 2018-06) so grouping/resampling by month preserves year boundaries (2018-01 ≠ 2019-01).  
- Less error‑prone than grouping by .dt.month (which loses year).  

Docs: [Pandas Period](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-period-aliases)

Step 2: Monthly aggregations by user

In [19]:
# Aggregate calls data
# Sum total minutes and count number of calls per user per month
calls_monthly = calls.groupby(['user_id', 'period'], as_index=False).agg(
    total_minutes=('duration', 'sum'),
    num_calls=('call_id', 'count')
)

print(f"\n✓ Calls aggregated:")
print(f"  • User-months: {calls_monthly.shape[0]:,}")
print(f"  • Unique users: {calls_monthly['user_id'].nunique()}")
print(f"  • Unique months: {calls_monthly['period'].nunique()}")


✓ Calls aggregated:
  • User-months: 3,174
  • Unique users: 492
  • Unique months: 12


In [20]:
# Aggregate internet data
# Sum total MB used and count number of sessions
internet_monthly = internet.groupby(['user_id', 'period'], as_index=False).agg(
    total_mb=('mb_used', 'sum'),
    num_sessions=('session_id', 'count')
)

print(f"\n✓ Internet aggregated:")
print(f"  • User-months: {internet_monthly.shape[0]:,}")
print(f"  • Unique users: {internet_monthly['user_id'].nunique()}")
print(f"  • Unique months: {internet_monthly['period'].nunique()}")


✓ Internet aggregated:
  • User-months: 3,203
  • Unique users: 497
  • Unique months: 12


In [21]:
# Aggregate messages data
# Count number of messages sent
messages_monthly = messages.groupby(['user_id', 'period'], as_index=False).agg(
    num_messages=('message_id', 'count')
)

print(f"\n✓ Messages aggregated:")
print(f"  • User-months: {messages_monthly.shape[0]:,}")
print(f"  • Unique users: {messages_monthly['user_id'].nunique()}")
print(f"  • Unique months: {messages_monthly['period'].nunique()}")


✓ Messages aggregated:
  • User-months: 2,717
  • Unique users: 426
  • Unique months: 12


Step 3. Create active user-month matrix  


Why don't create a matrix for every user * month?
Pros:  
- Simplifies joins/visualisations and monthly billing logic because missing = zero is explicit.
- Small scale (e.g. 500 users * 12 months) is trivial in memory and makes reporting robust.

Cons / costs:  
- Doesn't work with big and sparse datasets
- May produce rows for periods where user was not subscribed (before reg_date or after churn) - that gives misleading zeros.

In [22]:
# Determine the full date range across all datasets
start_date = min(
    calls['call_date'].min(),
    internet['session_date'].min(),
    messages['message_date'].min()
).to_period('M')
end_date = max(
    calls['call_date'].max(),
    internet['session_date'].max(),
    messages['message_date'].max()
).to_period('M')

# Create active periods for each user (registration to churn/data_end)
user_month_list = []

for _, user in users.iterrows():
    user_id = user['user_id']
    
    # User's active start = max(registration, data_start)
    reg_period = user['reg_date'].to_period('M')
    start_period = max(reg_period, start_date)
    
    # User's active end = min(churn, data_end) or data_end if not churned
    if pd.notna(user['churn_date']):
        churn_period = user['churn_date'].to_period('M')
        end_period = min(churn_period, end_date)
    else:
        end_period = end_date

# Only create periods if start <= end (user was active during data period)
    if start_period <= end_period:
        active_periods = pd.period_range(start=start_period, end=end_period, freq='M')
        
        user_df = pd.DataFrame({
            'user_id': user_id,
            'period': active_periods
        })
        
        user_month_list.append(user_df)

# Combine all users
active_user_months = pd.concat(user_month_list, ignore_index=True)



In [23]:
# Start with complete user-month matrix and left join all usage data
# This preserves all user-months, filling with NaN where there's no activity
df_monthly = (active_user_months
    .merge(calls_monthly, on=['user_id', 'period'], how='left')
    .merge(internet_monthly, on=['user_id', 'period'], how='left')
    .merge(messages_monthly, on=['user_id', 'period'], how='left')
)

print(f"\n✓ Usage data merged")
print(f"  Shape after merge: {df_monthly.shape}")


✓ Usage data merged
  Shape after merge: (3216, 7)


In [24]:
# Fill NaN values with 0 (no activity = zero usage)
usage_cols = ['total_minutes', 'num_calls', 'total_mb', 'num_sessions', 'num_messages']
df_monthly[usage_cols] = df_monthly[usage_cols].fillna(0)

# Convert to appropriate integer types for counts
df_monthly[usage_cols] = df_monthly[usage_cols].astype(int)

print(f"✓ Missing values filled with 0 for usage columns")

✓ Missing values filled with 0 for usage columns


In [25]:
# Convert MB to GB and round up (company billing policy)
# Any partial GB usage is billed as a full GB
df_monthly['gb_used'] = np.ceil(df_monthly['total_mb'] / 1024).astype(int)

print(f"✓ Converted {df_monthly['total_mb'].sum():,.0f} MB to GB (rounded up)")
print(f"  Sample conversions:")
print(f"  • 500 MB → {np.ceil(500 / 1024).astype(int)} GB")
print(f"  • 1024 MB → {np.ceil(1024 / 1024).astype(int)} GB")
print(f"  • 2048 MB → {np.ceil(2048 / 1024).astype(int)} GB")
print(f"  • 2049 MB → {np.ceil(2049 / 1024).astype(int)} GB")

✓ Converted 55,303,669 MB to GB (rounded up)
  Sample conversions:
  • 500 MB → 1 GB
  • 1024 MB → 1 GB
  • 2048 MB → 2 GB
  • 2049 MB → 3 GB


In [26]:
# Merge user demographic and subscription data
df_monthly = df_monthly.merge(users, on='user_id', how='left')

print(f"✓ User information added")
print(f"  Columns added: {list(users.columns)}")

✓ User information added
  Columns added: ['user_id', 'age', 'churn_date', 'city', 'first_name', 'last_name', 'reg_date', 'tariff']


In [27]:
df_monthly.head()

,user_id,period,total_minutes,num_calls,total_mb,num_sessions,num_messages,gb_used,age,churn_date,city,first_name,last_name,reg_date,tariff
0,1000,2018-05,159,22,2253,5,22,3,52,NaT,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1,1000,2018-06,172,43,23233,49,60,23,52,NaT,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
2,1000,2018-07,340,47,14003,29,75,14,52,NaT,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
3,1000,2018-08,408,52,14055,29,81,14,52,NaT,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
4,1000,2018-09,466,58,14568,27,57,15,52,NaT,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
